In [ ]:

!pip install langchain sentence-transformers faiss-gpu llama-cpp-python chromadb


In [ ]:
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DataFrameLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [ ]:
## only run once
#!huggingface-cli download TheBloke/Llama-2-7b-Chat-GGUF llama-2-7b-chat.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False
#!huggingface-cli download TheBloke/Mistral-7B-v0.1-GGUF mistral-7b-v0.1.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


%cd /content/drive/Shareddrives/'DATA 298 Team 7'/Data/"Bertha's Data"/RedditAPI/

In [ ]:
%cd /content/drive/Shareddrives/'DATA 298 Team 7'/Data/Bertha_Data/RedditAPI/



%ls

/content/drive/Shareddrives/DATA 298 Team 7/Data/Bertha_Data/RedditAPI
 old-files/             RedditPRAW.ipynb               top100csvs/
 pw.txt                 reddit_q_db/                   top100_travel_subreddit.csv
 reddit_3308_db/        RedditRequsts.ipynb            top100_travel_subreddit.gsheet
'reddit_3308_db (1)'/   top1000_travel_subreddit.csv


In [ ]:
%cd /content/drive/Shareddrives/'DATA 298 Team 7'/Data/'Sonia Processed'/'Processed Data for Prototyping'/QA/


%ls

/content/drive/Shareddrives/DATA 298 Team 7/Data/Sonia Processed/Processed Data for Prototyping/QA
dolly_travel_df.csv  NER/  travel_QA.pkl  travel_QA_processed_with_topic.csv


In [ ]:
import os

all_top100s = pd.concat([pd.read_csv(i,index_col="Unnamed: 0") for i in os.listdir()])

In [ ]:
link = "/content/drive/Shareddrives/'DATA 298 Team 7'/Data/Bertha_Data/RedditAPI/top1000_travel_subreddit.csv"

In [ ]:
%pwd

'/content/drive/Shareddrives/DATA 298 Team 7/Data/Bertha_Data/RedditAPI/top100csvs'

In [ ]:
data = pd.read_csv("top1000_travel_subreddit.csv",index_col=['Unnamed: 0']).sort_values("ups",ascending=False).dropna().rename(columns={'link_flair_css_class':'category'})
data.shape

(823, 11)

In [ ]:
reddit_3308_df = pd.concat([all_top100s,data]).drop_duplicates()

In [ ]:
reddit_3308_df.head()

,title,selftext,creation_date,id,url,upvote_ratio,ups,downs,score,link_flair_css_class,comments,category
0,"/r/solotravel ""The Weekly Common Room"" - Gener...",This thread is for you to do things like\n\n* ...,2023-11-05 16:00:38,17opmbr,https://www.reddit.com/r/solotravel/comments/1...,1.00,2,0,2,Accommodation,"[""If anyone's in Ho chi Minh city this week (6...",NaN
1,Weekly Destination Thread - Rio de Janeiro,This week’s destination is **Rio de Janeiro**!...,2023-11-05 11:53:48,17ok4ty,https://www.reddit.com/r/solotravel/comments/1...,0.76,4,0,4,NaN,"['Sunset from Pão de Açúcar', 'One of my favou...",NaN
2,What is the worst poverty you have come across...,Those of us who have ventured outside of the d...,2023-11-11 07:35:39,17swvwb,https://www.reddit.com/r/solotravel/comments/1...,0.87,171,0,171,Question,['In Ethiopia I was on a bus in some random to...,NaN
3,"First-time solo traveling in Luang Prabang, a ...","Hi there, I recently traveled to Luang Prabang...",2023-11-11 09:03:52,17sypme,https://www.reddit.com/r/solotravel/comments/1...,0.95,32,0,32,NaN,['Luang Namntha is worth going to as a launch ...,NaN
4,Curing constant thought about a trip I had for...,I had a two+ month solo trip last year which w...,2023-11-11 06:47:51,17svwin,https://www.reddit.com/r/solotravel/comments/1...,0.84,36,0,36,Question,['One thing I\'ve learned is that emotions tel...,NaN


# Creating Vector DB
### index on questions column

In [ ]:
question_loader = DataFrameLoader(reddit_3308_df, page_content_column="selftext")
question_data = question_loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=400,
                                          chunk_overlap=20)
texts = splitter.split_documents(question_data)

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2")

## Chromadb


In [ ]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


In [ ]:
%pwd

'/content/drive/Shareddrives/DATA 298 Team 7/Data/Bertha_Data/RedditAPI'

In [ ]:
from langchain.vectorstores import Chroma

chromadb_index = Chroma.from_documents(
    texts, embedding_function, persist_directory='./input'
)

In [ ]:
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline

retriever = chromadb_index.as_retriever()

model_id = "databricks/dolly-v2-3b" #my favourite textgeneration model for testing
task="text-generation"

hf_llm = HuggingFacePipeline.from_model_id(
    model_id=model_id,
    task=task,
    model_kwargs={
        "temperature": 0,
        "max_length": 256
    },
)

document_qa = RetrievalQA.from_chain_type(
    llm=hf_llm, chain_type="stuff", retriever=retriever
)

response = document_qa.run("Can I buy a neck pillow at the airport?")

display(response)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/450 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/819 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.68G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [ ]:
db = FAISS.from_documents(texts, embeddings)

db.save_local("reddit_3308_db")

In [ ]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs = {'device': 'cpu'})

q_db = FAISS.load_local("reddit_3308_db", embeddings)

### user queries can be matched with similar existing questions in database, retrieve thread comments

In [ ]:
docs = q_db.similarity_search("eating in san jose,california")
for d in docs:
  print(d.metadata['title'])
  print(d.metadata['comments'])
  print()

18 days in Mexico (looong post)
['Loved the notes for each place. Will be saving this for future reference. Thanks so much for the write up and the detail.', ">At some point, a guy with a French accent, sweaty and clearly upset, asks me for help. He said he was attacked by 3 Mexicans at the ATM and they took all his money and his cards. His hotel won't allow him to make a long-distance call to France because it will cost them too much and he asks me if I can lend him some money to go to a call center in centro and make a call to France from there.\n\nWe probably met the same guy. In August, I was walking down Avenida Tulum from ADO bus station to my hostel in Centro and a French dude asked for money after telling me a similar story. I also said no.\n\nIt wasn't in my plans either, but I also went to Puebla because a Puebla local I met in Guatemala convinced me to go there instead of Guanajuato before CDMX. Glad I did, really enjoyed it. Unfortunately the Cholula tunnels have been close

In [ ]:
pd.DataFrame({"matched_queries":[d.page_content for d in docs],"corresponding_responses":[d.metadata['comments'] for d in docs]})

,matched_queries,corresponding_responses
0,"My dilemma is this do I\n\n1. Go home, spendin...",['I don’t think I can give you any advice othe...
1,Planning a family vacation can be a rewarding ...,[]
2,Hello. Looking for a cruise line for a first t...,"[""Whatever you do, definitely talk to the crui..."
3,I am here to ask for some help. Me and my wife...,[]


In [ ]:
context = ""
for doc in docs:
    context += "Question: {}".format(doc.page_content)
    context += "Response: {}".format(doc.metadata["comments"])

In [ ]:
retriever = q_db.as_retriever(
    search_type="mmr",  # Also test "similarity"
    search_kwargs={"k": 20},
)

In [ ]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7db91f2c5930>, search_type='mmr', search_kwargs={'k': 20})

In [ ]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

llm = LlamaCpp(
  model_path="/content/drive/Shareddrives/DATA 298A Team 7/LLMs/llama-2-7b-chat.Q4_K_M.gguf",
  n_ctx=5000,
  n_gpu_layers=10,
  n_batch=512,
  f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
  callback_manager=callback_manager,
  verbose=True,
  )

ValidationError: 1 validation error for LlamaCpp
__root__
  Could not load Llama model from path: /content/drive/Shareddrives/DATA 298A Team 7/LLMs/llama-2-7b-chat.Q4_K_M.gguf. Received error Model path does not exist: /content/drive/Shareddrives/DATA 298A Team 7/LLMs/llama-2-7b-chat.Q4_K_M.gguf (type=value_error)

In [ ]:
llm(
    "Question: What can I do in South Korea this december around Christmas Time? If theres time Id like to visit another Asian country. Respond in less than 10 words. Answer:"
)

 Visit Seoul's festive markets, try Korean BBQ, and consider a trip to Japan! 

" Visit Seoul's festive markets, try Korean BBQ, and consider a trip to Japan! "

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)


In [ ]:
from langchain.chains.question_answering import load_qa_chain

# Prompt
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Make a disclaimer that your cited sources are from Reddit.
Always say "thanks for asking!" at the end of the answer and wish the user a safe travel.

{context}
Question: {question}
Answer:{key}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["context", "question", "key", "summaries"],
    template=template,
)

In [ ]:
question = 'What are some interesting activities to do in Europe around Christmas time?'

docs = retriever.get_relevant_documents(question)
docs

In [ ]:
 # Chain
chain = load_qa_chain(llm, chain_type="stuff", prompt=QA_CHAIN_PROMPT, verbose=True)

#key:
key = [doc.metadata["comments"] for doc in docs]

# Run
chain({"input_documents": docs, "question": question, "key": key})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three sentences maximum and keep the answer as concise as possible. 
I'll be going on a solo trip to Europe. Will arrive in Germany on Dec 2nd and will also visit Austria and Czech Republic and will stay until the 28th. I'm having a little bit of trouble trying to figure out where to go on December 24/25/26th to pass the holidays. I know that most of Europe simply closes during those days, so I'd appreciate any suggestions for places/cities in which you can simply

- snow activities (no skiing or snowboarding) it could be snow tubing, snow mobiles, etc.
- hot springs or similar
- lakes/kayaking/water rafting/water related excursions
- not really looking to do any crazy hikes, as it is a pretty big group
- drinking w

Llama.generate: prefix-match hit


 The best destinations for winter activities around Christmas time in Europe are London and Madrid. These cities offer a lively atmosphere during the holiday season, with various events and activities to enjoy. However, keep in mind that many restaurants and tourist attractions will be closed on December 24th and 25th, so plan accordingly. Alternatively, Vienna is also a great choice, with beautiful Christmas markets and a lovely atmosphere during the holidays.
> Finished chain.

> Finished chain.


{'input_documents': [Document(page_content="I'll be going on a solo trip to Europe. Will arrive in Germany on Dec 2nd and will also visit Austria and Czech Republic and will stay until the 28th. I'm having a little bit of trouble trying to figure out where to go on December 24/25/26th to pass the holidays. I know that most of Europe simply closes during those days, so I'd appreciate any suggestions for places/cities in which you can simply", metadata={'title': 'Where to go during Christmas in Europe?', 'creation_date': '2023-11-01 18:31:26', 'id': '17ls6ed', 'url': 'https://www.reddit.com/r/travel/comments/17ls6ed/where_to_go_during_christmas_in_europe/', 'upvote_ratio': 0.67, 'ups': 1, 'downs': 0, 'score': 1, 'category': 'question', 'comments': '[\'I would say either London or Madrid. You should be able to get reasonable flights, and both are lively and fun around Christmas time.\', \'Wow, people really do fly around the world for just a few days of travel. Woe this poor planet.\', \'